In [1]:
import pandas as pd
import numpy as np

import os
import sys
import warnings
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
 
from dataset import engine, RFM, Churn, Engagement, RFM_engage, RFM_churn, Reco, ROI
from models import CLFSwitcher, REGSwitcher, Transform, Pipe, parameters, reg_parameters, RecoSystem
from sklearn.model_selection import GridSearchCV

In [2]:
def train_roi(data):
	X = data.get_X()
	y = data.get_y()
	ct = Transform(data)
	# X, y = ct.get_Xy()

	pipeline = Pipe(task="REG").get_pipeline()
	def train(X, y, pipeline, parameters):
			grid_search = GridSearchCV(pipeline, reg_parameters, cv=5, n_jobs=12, return_train_score=True, verbose=1)
			# grid_search = pipeline
			grid_search.fit(X, y)
			return grid_search, grid_search.best_estimator_[-1]
	_, best_est = train(X, y, pipeline, parameters)
	# print(best_est.predict(X))
	return best_est
roi = ROI(engine)
roi_est = train_roi(roi)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [ ]:
import sqlalchemy

roi = ROI(engine)
X = roi.get_X()

pred = roi_est.predict(X)
data = roi.df
data[['clicks', 'leads', 'orders']] = pred
data[['clicks', 'leads', 'orders']] = data[['clicks', 'leads', 'orders']].astype(int)
# data['c_date'] = data['c_date'].dt.to_period("M").astype(str) 
data = data[data['c_date'] >= data['c_date'].max() - relativedelta(months=11)] \
	.groupby([data['c_date'].dt.to_period("M")]) \
	.agg({"clicks": ["sum"], "leads": ["sum"], "orders": ["sum"]}).reset_index()
data.columns = data.columns.get_level_values(0)
data['c_date'] = data['c_date'].astype(str)
data = data.rename(columns={'c_date': 'Campaign Month'})

data['clicksColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]
data['leadsColor'] = ["hsl(296, 70%, 50%)" for i in range(len(data))]
data['ordersColor'] = ["hsl(97, 70%, 50%)" for i in range(len(data))]
data = data.to_dict(orient='records')
data

[{'Campaign Month': '2024-01',
  'clicks': 15,
  'leads': 4,
  'orders': 0,
  'clicksColor': 'hsl(229, 70%, 50%)',
  'leadsColor': 'hsl(296, 70%, 50%)',
  'ordersColor': 'hsl(97, 70%, 50%)'},
 {'Campaign Month': '2024-02',
  'clicks': 77,
  'leads': 24,
  'orders': 8,
  'clicksColor': 'hsl(229, 70%, 50%)',
  'leadsColor': 'hsl(296, 70%, 50%)',
  'ordersColor': 'hsl(97, 70%, 50%)'},
 {'Campaign Month': '2024-03',
  'clicks': 151,
  'leads': 37,
  'orders': 22,
  'clicksColor': 'hsl(229, 70%, 50%)',
  'leadsColor': 'hsl(296, 70%, 50%)',
  'ordersColor': 'hsl(97, 70%, 50%)'},
 {'Campaign Month': '2024-04',
  'clicks': 39,
  'leads': 6,
  'orders': 8,
  'clicksColor': 'hsl(229, 70%, 50%)',
  'leadsColor': 'hsl(296, 70%, 50%)',
  'ordersColor': 'hsl(97, 70%, 50%)'},
 {'Campaign Month': '2024-05',
  'clicks': 77,
  'leads': 12,
  'orders': 16,
  'clicksColor': 'hsl(229, 70%, 50%)',
  'leadsColor': 'hsl(296, 70%, 50%)',
  'ordersColor': 'hsl(97, 70%, 50%)'},
 {'Campaign Month': '2024-06',
  '